In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# 1. Tokenization (15 pts)

How does one represent textual input to language models? One strategy that we have seen is to split up words on spaces, e.g.,

> This is an example.

> [This, is, an, example],

but this fails when unseen words appear at test time, e.g.,

> We named our son nwonkun.

> [We, named, our, son, \<unk\>] (5 tokens).

One solution to this problem is to use character-level tokens

> [W, e, _, n, a, m, e, d, _, o, u, r, _, s, o, n, _, n, w, o, n, k, u, n]

(24 tokens, if I counted right), but now the number of tokens required to encode a sentence has increased a *lot*.

## 1.1 Byte-pair encodings and sub-word tokenization (5 pts)

[Byte-pair encodings (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding) are a clever middle ground for the tokenization problem.
Starting from a character-level tokenization, iteratively combine the most common bigrams (token pairs) into their own tokens.
For example, the most common bigrams from the previous example are "_n" and "on". Breaking the tie arbitrarily and creating a new token "_n" we now have

> [W, e, _n, a, m, e, d, _, o, u, r, _, s, o, n, _n, w, o, n, k, u, n]

reducing the token count to 22. Iteratively applying this rule, we can further reduce it to 20 tokens by adding the token "on", and so on. Each step of this algorithm greedily reduces the token count by the maximum amount.

This tokenization scheme, known as "sub-word tokenization" takes the best of both worlds: since the vocabulary still contains tokens for every byte, we never have to use the \<unk\> token, while still reducing the number of required tokens to encode a sequence. The more tokens you add, the shorter your sequence gets.

To decide which tokens to add to the vocabulary, we have to *train* our BPE tokenizer on a corpus.
In this section you will do just that.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train: str = str.join(" ", dataset["train"]["text"])[:pow(10, 6)]
test: str = str.join(" ", dataset["test"]["text"])[:pow(10, 6)]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
from itertools import chain, pairwise
from collections import Counter
from tqdm import tqdm

class Tokenizer:
    # The lookup list contains *byte groups*, represented as a tuples of ints.
    # The token ID for a byte group is its index in the list.
    vocab: list[tuple[int, ...]]

    def __init__(self, training_seq: str, vocab_size: int) -> None:
        # Initialize a lookup with single-byte groups
        self.vocab = [(i,) for i in range(pow(2, 8))]
        byte_seq = list(bytes(training_seq, "utf-8"))
        for i in tqdm(range(pow(2, 8), vocab_size)):
            """
            TODO: iteratively add the most common token pairs to the vocabulary.
            Advice: try using Counter and pairwise from the python std lib.
            """
            pairs = pairwise(byte_seq)
            counts = Counter(pairs)

            if not counts:
                break

            common, _ = counts.most_common(1)[0]

            if all(0 <= b < 256 for b in common):
                self.vocab.append(common)
            #else:
            #    print(f"Skipping invalid pair: {common}")

            hu = []
            i = 0
            while i < len(byte_seq):
                if i < len(byte_seq) - 1 and (byte_seq[i], byte_seq[i + 1]) == common:
                    hu.append(len(self.vocab) - 1)
                    i += 2
                else:
                    hu.append(byte_seq[i])
                    i += 1
            byte_seq = hu

    def tokenize(self, seq: str) -> list[int]:
        """
        TODO: convert a byte sequence into a token sequence by greedily adding
        the longest token that matches the rest of the sequence, e.g.,
        vocab = [a, aa, b]
        sequence = aaab
        token_seq = [1, 0, 2] NOT [0, 1, 2].
        """
        byte_seq = list(bytes(seq, "utf-8"))
        token_seq = []

        i = 0
        while i < len(byte_seq):
            for length in range(len(self.vocab), 0, -1):
                token = tuple(byte_seq[i:i + length])
                if token in self.vocab:
                    token_seq.append(self.vocab.index(token))
                    i += length
                    break
            else:
                raise ValueError("No matching token found")
        return token_seq

    def detokenize(self, token_seq: list[int]) -> str:
        # TODO: convert a token sequence into a byte sequence.
        byte_seq = list(chain.from_iterable(self.vocab[token] for token in token_seq))
        return bytes(byte_seq).decode("utf-8")

train_data = train[:10000]
bigrams = list(pairwise(bytes(train_data, "utf-8")))
bigram_counts = Counter(bigrams)
print("Most common bigrams in the training data:")
print(bigram_counts.most_common(10))
tokenizer = Tokenizer(train_data, vocab_size=500)

print("Some of our new tokens:")
for token in tokenizer.vocab[-10:]:
    print(repr(bytes(token).decode("utf-8")))

Most common bigrams in the training data:
[((101, 32), 301), ((32, 116), 249), ((115, 32), 231), ((116, 104), 203), ((104, 101), 173), ((32, 97), 167), ((100, 32), 152), ((110, 32), 145), ((101, 114), 139), ((97, 110), 124)]


100%|██████████| 244/244 [00:00<00:00, 249.03it/s]

Some of our new tokens:
'ga'
'be'
'su'
'ki'
'ug'
'as'
'av'
'ul'
'bo'
'we'


As a sanity check, your implementation should be able to compress the training set to ~40-50% of its original size.
You should notice that the test set compression does not perform as well. This is because the distribution of bigrams in the test set does not exactly match the that of the train set. This gets worse the further your test set distribution is from your training set.

In [ ]:
# Do not edit this code cell
test_data = test[:10000]
train_bytes_len = len(bytes(train_data, "utf-8"))
train_token_len = len(tokenizer.tokenize(train_data))
print(f"Compressed train set to {train_token_len / train_bytes_len * 100:.0f}% original size")
test_bytes_len = len(bytes(test_data, "utf-8"))
test_token_len = len(tokenizer.tokenize(test_data))
print(f"Compressed test set to {test_token_len / test_bytes_len * 100:.0f}% original size")

assert train_data == tokenizer.detokenize(tokenizer.tokenize(train_data))
assert test_data == tokenizer.detokenize(tokenizer.tokenize(test_data))

Compressed train set to 63% original size
Compressed test set to 66% original size


## 1.2 BPE performance on OOD text. (5 pts)

Explore how English-trained BPE performs on non-English text by downloading corpora from a few different languages and using your English-trained tokenizer. What do you find? Do the results match your expectations? For what langauges does the tokenizer struggle with the most? How might this impact society if everyone were to use your tokenizer?

Include your code, results, and discussion in new cells below.

Hint: we recommend you use `load_dataset` to fetch from HuggingFace with `streaming=True` to avoid huge downloads. You might want to take a look at the `oscar` dataset.

In [ ]:
from itertools import islice

langs = ["fr", "es", "zh", "ar"]
corp = {}

for i in langs:
    dataset = load_dataset("oscar", f"unshuffled_deduplicated_{i}", streaming=True, trust_remote_code=True)
    corp[i] = " ".join(item["text"] for item in islice(dataset["train"], 1000))

for i, corpus in corp.items():
    td = corpus[:10000]
    tl = len(tokenizer.tokenize(td))
    tb = len(bytes(td, "utf-8"))

    print(f" {i} = {tl / tb * 100:.0f}% original size")
    sample = tokenizer.tokenize(td[:200])
    #print(f"Sample : {sample}")
    detokenized_sample = tokenizer.detokenize(sample)
    #print(f"detokenized : {detokenized_sample}")


 fr = 74% original size
 es = 68% original size
 zh = 100% original size
 ar = 99% original size


## 1.3 Pitfalls of and alternatives to BPE (5 pts)

BPE tokenization sufferes from other issues as well. Due to the implementation of our BPE tokenizer, detokenizing a sequence of tokens then re-tokenizing it does not always recover the original sequence:
```
vocab = {a, aa, b}
tokens = [0, 1, 2]
detokenized = aaab
retokenized = [1, 0, 2]
```

Another issue is that some tokens that may have been prevalent during BPE training may not be present during language model training, leading to funky situations where the language model has not been trained to represent or output some tokens. See this paper for more information: https://arxiv.org/pdf/2405.05417.

Some NLP researchers think that we should move away from sub-word tokenization to get rid of these problems. Engage with this discussion by either
- Finding a paper that points out an issue with tokenization and propose your own solution for how you would fix it, or
- Finding a paper that proposes an alternative tokenization scheme (or way of processing text) and discuss the drawbacks of the proposed method.

Your response should be about a paragraph in length and link to a paper.

**This paper titled Byte Pair Encoding is Suboptimal for Language Model Pretraining (https://arxiv.org/pdf/2004.03720) addresses the issues associated with BPE. One issue is that it merges/splits words in a manner that may not be contextually correct, such as words with vital latin roots that it could miss ( something like Universal). Words that commonly appear in threes cause a pairing between two words and the third intermediate token will be a standalone in the vocabularily thats rarely used. BPE also faces a problem where common affixes are merged with other tokens that they shouldn't be, making it sound less coherent.**

# 2. Generation Algorithms (35 pts + 15 pts BONUS)

In this problem, we will implement several common decoding algorithms and test them with the GPT-2 Medium model.

Given the class below, we will fill in each of the method stubs. You may create additional helper methods as well to make components re-usable.

**You are not allowed to use the generate() function in the transformers library. You can only use the model's forward() method to retrieve final layer logits**

In addition to the methods we ask you to implement, which are:
- Greedy decoding
- Temperature Sampling
- Nucleus Sampling

You will choose ONE of the following sampling algorithms to implement as well (make sure to add your own method, since we do not provide one by default):
- Typical Sampling ([Meister et al. (2022)](https://arxiv.org/abs/2202.00666))
- Eta Sampling ([Hewitt et al. (2022)](https://arxiv.org/abs/2210.15191))

Points for this question will be distributed as follows:

- 5-10 points for implementing each decoding algorithm
- 5 points for implementing the generate() function (you will make this incrementally through each sub-part)
- 5 points for filling out the table with list of tokens (see instructions below)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import Optional

class LM():
  def __init__(self, model_name: str = "openai-community/gpt2-medium"):
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.model = AutoModelForCausalLM.from_pretrained(model_name)
    self.model.eval()

  def greedy_decoding(self, prompt: str, max_length: int = 64) -> str:
    """
    TODO:

    Implement greedy decoding, in which we use the highest
    probability token at each decoding step
    """
    ids = self.tokenizer(prompt, return_tensors="pt").input_ids
    out = ids.clone()

    for x in range(max_length):
        logits = self.model(input_ids=out).logits
        nids = torch.argmax(logits[:, -1, :], dim=-1)
        out = torch.cat([out, nids.unsqueeze(-1)], dim=-1)
        if nids.item() == self.tokenizer.eos_token_id:
            break

    return self.tokenizer.decode(out[0], skip_special_tokens=True)

  def temperature_sampling(self, prompt: str, temperature: float = 1.0, max_length: int = 64) -> str:
    """
    TODO:

    Implement temperature sampling, in which we sample
    from the output distribution at each decoding step,
    with a temperature parameter to control the "peakiness"
    of the output distribution
    """
    ids = self.tokenizer(prompt, return_tensors="pt").input_ids
    out = ids.clone()

    for x in range(max_length):
        logits = self.model(input_ids=out).logits
        scaled_logits = logits[:, -1, :] / temperature
        probs = torch.softmax(scaled_logits, dim=-1)
        nids = torch.multinomial(probs, num_samples=1)
        out = torch.cat([out, nids], dim=-1)
        if nids.item() == self.tokenizer.eos_token_id:
            break

    return self.tokenizer.decode(out[0], skip_special_tokens=True)

  def nucleus_sampling(self, prompt: str, p: float = 0.9, max_length: int = 64, temperature: float = 1.0) -> str:
    """
    TODO:
    Implement nucleus sampling, in which we
    sample from a subset of the vocabulary
    at each decoding step
    Note: There is also a temperature parameter here
    """
    ids = self.tokenizer(prompt, return_tensors="pt").input_ids
    out = ids.clone()

    for x in range(max_length):
        logits = self.model(input_ids=out).logits
        scaled_logits = logits[:, -1, :] / temperature
        probs = torch.softmax(scaled_logits, dim=-1)
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        allprobs = torch.cumsum(sorted_probs, dim=-1)

        inRange = allprobs <= p
        if not inRange.any():
            nids = sorted_indices[0, 0].unsqueeze(0)
        else:
            nucleus_indices = sorted_indices[inRange]
            nucleus_probs = sorted_probs[inRange]

            nids = nucleus_indices[torch.multinomial(nucleus_probs, num_samples=1)]

        out = torch.cat([out, nids.unsqueeze(0)], dim=-1)
        if nids.item() == self.tokenizer.eos_token_id:
            break

    return self.tokenizer.decode(out[0], skip_special_tokens=True)

  def typical_sampling(self, prompt: str, t: float = 0.9, max_length: int = 64, temperature: float = 1.0) -> str:

    ids = self.tokenizer(prompt, return_tensors="pt").input_ids
    out = ids.clone()

    for x in range(max_length):
        logits = self.model(input_ids=out).logits
        scaled_logits = logits[:, -1, :] / temperature
        probs = torch.softmax(scaled_logits, dim=-1)

        entropy = -(probs * probs.log()).sum(dim=-1, keepdim=True)
        conv = -probs.log() / entropy
        sorted_conv, sorted_indices = torch.sort(torch.abs(conv - t), descending=False)
        truncated_probs = probs[0, sorted_indices[0]]

        normalized_probs = truncated_probs / truncated_probs.sum()
        sampled_index = torch.multinomial(normalized_probs, num_samples=1).item()
        next_token = (sorted_indices[0][sampled_index]).unsqueeze(0).unsqueeze(0)
        out = torch.cat([out, next_token], dim=-1)

        if next_token.item() == self.tokenizer.eos_token_id:
            break

    return self.tokenizer.decode(out[0], skip_special_tokens=True)

  def generate(self,
               prompt: str,
               temperature: float = 1.0,
               p: Optional[float] = None) -> str:
      """
      TODO:

      Route to the appropriate generation function
      based on the arguments
      HINT: What parameter values should map to greedy decoding?
      """
      max_length = 64
      if p is not None:
          return self.nucleus_sampling(prompt, p=p, temperature=temperature, max_length=max_length)
      elif temperature < 0.5:
          return self.greedy_decoding(prompt, max_length=max_length)
      else:
          return self.temperature_sampling(prompt, temperature=temperature, max_length=max_length)


  def greedyTop10(self, prompt: str) -> list[tuple[str, float]]:
      input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
      logits = self.model(input_ids=input_ids).logits[:, -1, :]
      topkprob, topk = torch.topk(torch.softmax(logits, dim=-1), 10)

      tokens = []
      for prob, idx in zip(topkprob[0], topk[0]):
          try:
              decoded = self.tokenizer.decode([idx])
              tokens.append((decoded, prob.item()))
          except UnicodeDecodeError:
              continue

      return tokens

  def typicalTop10(self, prompt: str, t: float = 0.9, temperature: float = 1.0) -> list[tuple[str, float]]:
        """
        Implements Typical Sampling's top 10 tokens at the first decoding step.
        """
        input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
        logits = self.model(input_ids=input_ids).logits[:, -1, :] / temperature
        probs = torch.softmax(logits, dim=-1)

        entropy = -(probs * probs.log()).sum(dim=-1, keepdim=True)
        conv = -probs.log() / entropy
        deviation = torch.abs(conv - t).squeeze()
        sorted_indices = torch.argsort(deviation)

        top10i = sorted_indices[:10]
        tenprob = probs[0, top10i]
        tentoken = [(self.tokenizer.decode(idx), prob.item()) for idx, prob in zip(top10i, tenprob)]

        return tentoken

  def tempTop10(self, prompt: str, temperature: float = 1.0) -> list[tuple[str, float]]:
      input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
      logits = self.model(input_ids=input_ids).logits[:, -1, :] / temperature
      topkprob, topk = torch.topk(torch.softmax(logits, dim=-1), 10)

      tokens = []
      for prob, idx in zip(topkprob[0], topk[0]):
          decoded = self.tokenizer.decode([idx])
          if decoded.strip():
              tokens.append((decoded, prob.item()))
      return tokens

  def nucleusTop10(self, prompt: str, p: float = 0.9, temperature: float = 1.0) -> list[tuple[str, float]]:
      input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
      logits = self.model(input_ids=input_ids).logits[:, -1, :] / temperature
      probs = torch.softmax(logits, dim=-1)
      sorted_probs, sorted_indices = torch.sort(probs, descending=True)

      inRange = (torch.cumsum(sorted_probs, dim=-1) <= p)
      topkprob = sorted_probs[inRange][:10]
      topk = sorted_indices[inRange][:10]

      tokens = []
      for prob, idx in zip(topkprob, topk):
          decoded = self.tokenizer.decode([idx])
          if decoded.strip():
              tokens.append((decoded, prob.item()))
      return tokens


lm = LM()
prompt = "Once upon a time in a land far far away,"

print("greedy:", lm.greedyTop10(prompt))
print("temperature (t=1.0):", lm.tempTop10(prompt))
print("nucleus (p=0.9):", lm.nucleusTop10(prompt))
print("typical sampling:", lm.typicalTop10(prompt, t=0.9, temperature=1.0))

greedy: [(' there', 0.21611085534095764), (' a', 0.16582567989826202), (' the', 0.06953583657741547), (' in', 0.03848272189497948), ('\n', 0.03528774157166481), (' I', 0.0322796106338501), (' an', 0.01550489105284214), (' when', 0.009391860105097294), (' two', 0.009228527545928955), (' you', 0.009130480699241161)]
temperature (t=1.0): [(' there', 0.21611085534095764), (' a', 0.16582567989826202), (' the', 0.06953583657741547), (' in', 0.03848272189497948), (' I', 0.0322796106338501), (' an', 0.01550489105284214), (' when', 0.009391860105097294), (' two', 0.009228527545928955), (' you', 0.009130480699241161)]
nucleus (p=0.9): [(' there', 0.21611085534095764), (' a', 0.16582567989826202), (' the', 0.06953583657741547), (' in', 0.03848272189497948), (' I', 0.0322796106338501), (' an', 0.01550489105284214), (' when', 0.009391860105097294), (' two', 0.009228527545928955), (' you', 0.009130480699241161)]
typical sampling: [(' an', 0.01550489105284214), (' when', 0.009391860105097294), (' two

In [ ]:
GPT2LM = LM("openai-community/gpt2-medium")

For each sampling algorithm you implement, fill out this table, in which you will list the top 10 highest probability tokens **at the first decoding step** in a comma separated list. For algorithms like nucleus sampling where you perform some kind of truncation/re-distribution of the output distribution, do the truncation/re-distribution first, and then sort the vocabulary by probability to complete the table.

For this and all questions below, use the following prompt:


**"Once upon a time in a land far far away, "**

Note: Use the default value for `max_length` for all questions below.

| **Decoding Algorithm** | **10 Highest Probability Tokens** |
|------------------------|-----------------------------------|
| Greedy                 | [(' there', 0.21611085534095764), (' a', 0.16582567989826202), (' the', 0.06953583657741547), (' in', 0.03848272189497948), ('\n', 0.03528774157166481), (' I', 0.0322796106338501), (' an', 0.01550489105284214), (' when', 0.009391860105097294), (' two', 0.009228527545928955), (' you', 0.009130480699241161)] |
| Temperature (t=1.0)    | [(' there', 0.21611085534095764), (' a', 0.16582567989826202), (' the', 0.06953583657741547), (' in', 0.03848272189497948), ('\n', 0.03528774157166481), (' I', 0.0322796106338501), (' an', 0.01550489105284214), (' when', 0.009391860105097294), (' two', 0.009228527545928955), (' you', 0.009130480699241161)]  |
| Nucleus (p=0.9)        | [(' there', 0.21611085534095764), (' a', 0.16582567989826202), (' the', 0.06953583657741547), (' in', 0.03848272189497948), ('\n', 0.03528774157166481), (' I', 0.0322796106338501), (' an', 0.01550489105284214), (' when', 0.009391860105097294), (' two', 0.009228527545928955), (' you', 0.009130480699241161)]|
| Typical/Eta            | [(' an', 0.01550489105284214), (' when', 0.009391860105097294), (' two', 0.009228527545928955), (' you', 0.009130480699241161), (' I', 0.0322796106338501), (' many', 0.007748258300125599), ('\n', 0.03528774157166481), (' in', 0.03848272189497948), (' my', 0.006503292825073004), (' one', 0.006039347033947706)]|

## 2.1 Greedy Decoding (5 points)

First, implement the most simple decoding method of greedy decoding. Here, at each decoding time step, simply use the highest probability token. Note that you'll need to adjust the generate function so that a specific temperature value will map to greedy decoding (what should that value be?).

Use the prompt given above to test your implementation. What do you notice?

In [ ]:
lm = LM()
prompt = "Once upon a time in a land far far away,"
output = lm.greedy_decoding(prompt)
print("Greedy output:", output)

Greedy output: Once upon a time in a land far far away, there lived a man named Simeon. He was a wise man, and he knew the secrets of the universe. He knew that the universe was made of many worlds, and that each world was made of a single substance. He knew that each substance was made of a single substance, and that each substance was made


**Greedy Decoding Output: Once upon a time in a land far far away,  a young man named   was born.  He was a   tall,   beautiful,   and   strong  man.  He was also a   very   strong  man.  He was also a**   

**I notice it's very repetitive, and doesn't sound very fluent**

##2.2 Temperature Sampling (10 pts)

Sometimes (a lot of the time?), we don't actually just want the highest probability token at each time step. Why might this be the case?

**This is because using the most likely words every time will limit the scope of your language, and you will be using the same words repetitively since they will all have high probabilities with each other**

To adjust for this, we often use sampling algorithms instead of greedy decoding. However, there are many ways we can go about sampling.

First, implement temperature sampling. Recall that the temperature parameter adjusts the "randomness" of the output at each time step. Here, you'll need to think about how to adjust the output distribution which you will do multinomial sampling from. Be careful about how you will handle very low (close to 0) temperatures.

Given the same prompt as above, test your implementation with the following temperature values: [0.3, 0.5, 0.7, 0.9, 1.1]. For each value, sample 3 outputs. What do you notice in terms of the differences between output sets across different temperature values?  

In [ ]:
lm = LM()
prompt = "Once upon a time in a land far far away,"
output = lm.temperature_sampling(prompt)
print(output)

Once upon a time in a land far far away, in a land far far away, I once owned property with no workers compensated pension or emergency funds and I had a lady. And she worked for eight years and there was one principal who would develop out of her right arm when she wanted cement chips." (comment from The Graham Hancock Show until the early 90s) I


In [ ]:
temperatures = [0.3, 0.5, 0.7, 0.9, 1.1]
results = {}

prompt = "Once upon a time in a land far far away,"
lm = LM()

for temp in temperatures:
    output = lm.generate(prompt, temperature=temp)
    results[temp] = {"Output": output}

for temp, data in results.items():
    print(f"Temperature: {temp}")
    print(f"Generated Output: {data['Output']}")
    print("\n\n")

Temperature: 0.3
Generated Output: Once upon a time in a land far far away, there lived a man named Simeon. He was a wise man, and he knew the secrets of the universe. He knew that the universe was made of many worlds, and that each world was made of a single substance. He knew that each substance was made of a single substance, and that each substance was made



Temperature: 0.5
Generated Output: Once upon a time in a land far far away, in a land far away, some people lived in a cave. There were no trees or shrubs. There were only rocks and rocks and rocks. And the rocks were red. The rocks were red. And the rocks were red. And the rocks were red. And the rocks were red. And the rocks were red



Temperature: 0.7
Generated Output: Once upon a time in a land far far away, I was told that the greatest of all fates was to befall me, and that he who should stand before me should encounter an unseen enemy whom I could never hope to defeat. I was told that I was to be the first to fall, an

| **Temperature** | **Output 1** | **Output 2** | **Output 3** |
|-----------------|--------------|--------------|--------------|
| 0.3             | Once upon a time in a land far far away, there lived a man named Simeon. He was a wise man, and he knew the secrets of the universe. He knew that the universe was made of many worlds, and that each world was made of a single substance. He knew that each substance was made of a single substance, and that each substance was made | Once upon a time in a land far far away, there lived a man named Simeon. He was a wise man, and he knew the secrets of the universe. He knew that the universe was made of many worlds, and that each world was made of a single substance. He knew that each substance was made of a single substance, and that each substance was made| Once upon a time in a land far far away, there lived a man named Simeon. He was a wise man, and he knew the secrets of the universe. He knew that the universe was made of many worlds, and that each world was made of a single substance. He knew that each substance was made of a single substance, and that each substance was made|
| 0.5             | Once upon a time in a land far far away, there was a man who ran a small tavern in a small town, and he had a friend who was in love with a lady in a neighboring town. The friend was a young man, and the friend was a good friend of the man's. The friend had given the young man a very large sum of money, | Once upon a time in a land far far away, a young man was born named Kirito. He was a very shy and timid boy, but he was determined to become a great adventurer. He was raised in a village with a large, beautiful, and beautiful woman named Yukino. When he was young, he was very curious about the world around him |  Once upon a time in a land far far away, there lived a young man named Gavriel. He was tall and handsome, and he had a bright smile. He was also a soldier, and he was on his way to the front lines of the war.Gavriel had always dreamed of becoming a knight. He had been born into a noble family            |
| 0.7             | Once upon a time in a land far far away, in a land where stars and planets danced in the nights, a band of heroes became heroes in the land of legend. They are the Pandaur Warriors, the champions of the free world. They are the heroes of the land of legend! |  Once upon a time in a land far far away, a young man crossed the river to live in peace with the natives. As he lived there, he heard of the desire of the people of the forest for a king rather than a king who had been slain by bandits. So he took his journey and came upon a village, near which he saw a large stone dragon.| Once upon a time in a land far far away, a young girl was being experimented with. Her name was Corrine. She had the power to see into the future and could use that power to change the future. The princess, Corrine had been experimented on by Carver, a former high ranking government official. She was given the ability to be an immortal. The |
| 0.9             | Once upon a time in a land far far away, Historia Barcelona was not alive, but was in a continuing state. They had a mayor and a mayoress, not to mention a newspaper that was published there, ditto sports books for fashion and celebrity gossip. At a time when pirate radio obsession was a thing, this was the kind of music you could hear.| Once upon a time in a land far far away, a man by the name of Mihi ran a simple game. He gave mud pies to children, and he played each. The children loved children, and played and played until school was out. Then, one day, there was the strong man's poison and the girl's antidote to the poison. The cure was| Once upon a time in a land far far away, a sword knight began his training to become a sword master. The sword knight could augment his own strength at will without using a person as his core for physical training. Able to lift a sword of limitless power and extend it to wherever he desires at will, he could quickly offend even the most vicious mercenaries of all|
| 1.1             | Once upon a time in a land far far away, King Jork was trying to bribe and bribe some children, to take plots of land, or heads of great duchies. Well, He got 73 nobility up to said point! But madum da capŝ, It used to rain down before his big gr't mades'. Ice wish to push tools for | Once upon a time in a land far far away, 2 peaceful towns met. Leather rarely exists long enough for a country to spread either, so when the sole white collar resulting Ashton went on another strike formed…..and they both donned… balaclavas! Who knew? | Once upon a time in a land far far away, a king decided giving aquatic living creatures the things of nature was a good idea. This led to everything from lizards giving bats technology and the plaintiffs in our Privacy First lawsuit, the VROO state bordering something little more Mysticism. But quite a few predators of all these vast oceanic wildlife - mainly netted brown bears |

**The lower temperatures have much less to no variation, and are repitive. High temperatures have variation but don't make fluent sense, middle temperatures have a mix of both**

## 2.3 Nucleus Sampling (10 pts)

Originally published in [Holtzmann et al. (2021)](https://arxiv.org/abs/1904.09751), nucleus sampling was designed to address an issue that was especially prevalent in language models at the time.

This issue is the case of "neural text degeneration," where outputs from LMs would often degenerate into gibberish if a low probability token was ever decoded. To address this, nucleus (also known as top-p) sampling uses a hyperparameter, p, to control how big of a subset of the vocabulary we sample from at each step. For example, if p=0.9, we only sample from the subset of tokens that have a cumulative probability mass of 0.9 (after sorting by probability).

Implement nucleus sampling and then use the same prompt as above and test your implementation with the following p-values: [0.97, 0.95, 0.9, 0.8, 0.7]
What do you notice across outputs?

In [ ]:
vals = [0.97, 0.95, 0.9, 0.8, 0.7]
prompt = "Once upon a time in a land far far away,"
lm = LM()

for p in vals:
    output = lm.generate(prompt, p=p, temperature=1.0)
    print(f"p={p}\n{output}")
    print("\n\n")

p=0.97
Once upon a time in a land far far away, important messages were communicated to all land. Though extraordinary, these are unheard of in this time. One of these is written in the ancient Book of Legends. This book depicts an event that took place during a rising and town fell; In the sky was growing kindle, trapped between the stars and scorching the land.



p=0.95
Once upon a time in a land far far away, every living being was turned into a doll, a doll in an otherwise naturalistic manner but utilizing an artificial intelligence with the ability to grow and function naturally again, even if for only a short time.

Creation

Grady Stacy was created by Kristopher Ryba for Marvel's podcast in 2012, originally



p=0.9
Once upon a time in a land far far away, upon a soil of glass, there lived a man named Mengele. There was one day when he asked me whether I was a Jew or not. I told him I didn't know, and it surprised him. He replied: "When you give us money, please don't mention y

| **p** | **Output 1** | **Output 2** | **Output 3** |
|----------|--------------|--------------|--------------|
| 0.97     | Once upon a time in a land far far away, we were pulled into a new empire and tried to adapt our military plans to the new culture we were raised in by a spy named Leo. I understand you are a bit reluctant but I will trust you to adapt. I'll keep the word of the man and your spinelessness won't prevent me from deducing the | Once upon a time in a land far far away, when we got to ski or snowboard, many more did the prep than we did. That's because it's the equivalent of one climbing season per person. This was 100 years ago. A soccer cup has added another 40 years to our lives.| Once upon a time in a land far far away, there was an elf who took to politics. Someone who fought against the warlords that lived under it. An elf-writer living in a tavern running a magical journal. But then one day, she felt something her editor wouldn't tell her. She whispered to the manager of her pub, asking them for feedback on her|
| 0.95     | Once upon a time in a land far far away, by my fathers name Rawdydom , I was a courier and hard of hearing, too young, Though much or little wistful, I have sat there Many a week, usually droning away,Sometimes more entranced than ever, Refreshing with  | Once upon a time in a land far far away, There came a maiden who said: Your art thou, stranger, with a sly heart? The dueling voice is not of mine, But instead you eluded me, thou demon,I had sworn to put you to death, But I strove not. | Once upon a time in a land far far away, this question was posed to Ah's teacher of law. This mostly just rubbed everyone the wrong way. Over time, each learner's teacher struggled to prove that there wasn't something strange about dealing with a sage, and eventually, a statue of Ah fell from Mt. Tai. Even so, even those who|
| 0.9      |Once upon a time in a land far far away, Manaa was about to escape from the greatest threat that Manaa ever encountered. However, she realized that the game had ended. A small incantation appeared over her ear, which made her feel unwell. She was forced to leave in the middle of the night.Her mother, whose business is travelling             | Once upon a time in a land far far away, an evil sorcerer named Bane entered the Far Shores with his evil are alive called of evil. Bane's plan was simple, to regain the power of the Black Prince without having to resort to evil. Having well known the Black Prince's weakness, Bane travelled the world seeking to gain his attention. His first target was Warren| Once upon a time in a land far far away, the Dragon party reached White Rock Camp in the Karak Orphanage. And Yoki attended his orders to kill King Bodo Aalds without question. However the full truth of events transpired, only to set Yoki on a new course of action once he returned to Karak. A king   |
| 0.8      | Once upon a time in a land far far away, there lived a man named Erma. She was a bright and pretty girl. She was an amazing singer and dancer. And she loved her goddess. It wasn't much, but it was something. And when the god of thunder, Valamir, came down from the heavens, he was delighted. He asked her| Once upon a time in a land far far away, the ancient king died and his last royal corpse was laid to rest. Many years later, another prince became king. He, too, died and his final royal corpse was laid to rest. Finally, a third prince who ruled over an empire which was split between six kingdoms appeared and seized the throne. The king wished to | Once upon a time in a land far far away, my mind had wander into the wilds of mystery and violence, and I was made to kneel before a great demon. In those days there were no heroes of a moral nature; The only question was which of my old men had the courage to oppose this villain With a sword at his |
| 0.7      | Once upon a time in a land far far away, there lived a boy named Teemo. The boy who was like many others, loved spending time with his friends. In the age of Diablo, the village he lived in was besieged by an invading orc. Teemo was so tired of fighting that he decided to rest. This did not go unnoticed by his friends, and| Once upon a time in a land far far away, there lived a strong man, who loved his land very much. He was wise, very kind, and cared very much for his people. And he was a good shepherd. So the man named Simcha lived a long time, and then his land was scattered over the lands of the gods. He fed his people,| Once upon a time in a land far far away, on a distant island, two men in their thirties were fighting. As they were on the verge of killing each other, one of them pulled out a sword and thrust it into the other's chest. The one who lost his arm and lost his life was no longer able to speak. He  |

**With higher P values, the outputs use more complex language. Lower p-values sound more like how a child would tell a story, whereas higher p has a more diverse language and structure (such as poem format, which i had to cut out to put into the table here)**

## 2.4 More variations on decoding algorithms (10 pts)

Nucleus sampling was definitely not the end of the road in terms of new decoding algorithms. Even in the past few years, new decoding algorithms have been proposed to address some limitations of existing algorithms.

Two in particular are:
- Typical Sampling ([Meister et al. (2022)](https://arxiv.org/abs/2202.00666))
- Eta Sampling ([Hewitt et al. (2022)](https://arxiv.org/abs/2210.15191))

For this question, CHOOSE ONE of the two algorithms presented above. Below, please describe in a few sentences what your chosen algorithm does in a novel way and the broad motivation behind it. Along with this description, present 3 sampled outputs for the same prompt as above (you can use one hyperparameter value for all of these).



**Typical sampling seeks to see how "typical" or expected a token is. It takes the -log of the probaility and normalizes it, and then compares that to the distribution to find how unexpected it is. The controlling variable here decides how far from the distribution it is willing to look, with a higher value meaning more diverse and less typical words/tokens**

In [ ]:
prompt = "Once upon a time in a land far far away,"
lm = LM()

t = 0.9
for i in range(3):
    output = lm.typical_sampling(prompt, t=t, temperature=1.0)
    print(f" {i}: {output}\n\n\n")

 0: Once upon a time in a land far far away, the pursuit of withered dreams, while fame had been a fawning your share among the untutored leaders, there walked a-- native of enchantments places, a wailing wanderer. Woke, and out he lighted on the cemetery, and in that grave lay the body. Alysanli,



 1: Once upon a time in a land far far away, surrounded by deep sea, where everywhere there was natural (not always formless) Water, filled the land with Cloaks so thick with dangerous Qi and under the Cloaks were reefs such as a river.

The Cloaks comprised of mobile Zithers and their cables held down these fierce true Water Arrows all night



 2: Once upon a time in a land far far away, I was teaching exactly that thing everyone is feeding folks on campus. She repointed her pupils, for she knew her class & incautiously speaks recklessly of the august titles of 'reeowned wombs' & 'so popular for whiskering towards a wife and bells' & hisd lexic magisterily





| Decoding Method | **Output 1** | **Output 2** | **Output 3** |
|-----------------|--------------|--------------|--------------|
| Typical               | Once upon a time in a land far far away, the albino unnaM personhood movement entered a process of self-discovery where it could "in both shape and color" voluntarily before becoming a voice, more so than challenges enveloping start-up film states. The movement centred around fountain penist Gabrielle Elliot's casting, which was            | Once upon a time in a land far far away, a very brave woman called Marigold fell and she cried profusely saying, 'The Road has paved to your door'; and she brought me some lady from Algarve which was a witch, and it was full of witchcraft. – Prince Daniel, Auden four quatre formato 1889 If you | Once upon a time in a land far far away, where there was only the world known we call earth and sky, then the leader of mankind found himself confronted with a surprise. He was faced time and again with the question of who he should be. Now is the time to choose. It's an absolute certainty. |

## 2.5 BONUS (Up to 15 pts)

Can you find a prompt where the continuations do not differ much across multiple sampling strategies, even when we use high temperatures or high p values? (Hint: Think about overfitting)

In [ ]:

prompt = [
    "The capital of the United States is"
]

lm = LM()
temperature = 1
p = 0.8

for i in prompt:
    print("Greedy:\n")
    print(lm.greedy_decoding(i))
    print("temp:")
    print(lm.temperature_sampling(i, temperature=temperature))
    print("\nNucleus:\n")
    print(lm.nucleus_sampling(i, p=p, temperature=temperature))
    print("Typical")
    print(lm.typical_sampling(i, t=0.9, temperature=temperature))
    print("\n\n")


Outputs:

Greedy:
The capital of the United States is Washington, D.C.

The capital of the United States is Washington, D.C.

The capital of the United States is Washington, D.C.

The capital of the United States is Washington, D.C.

The capital of the United States is Washington, D.

-------------------------

Temperature:
The capital of the United States is the Republican capital, and it's a large NFL town that's worth a lot of money while playing its full role in society, according to the research company of University College. Ohio has no equal financial passion, argues William Bryson, the lead author of the study, who explains Americans choose their national capital city on "

-------------------------

Nucleus:
The capital of the United States is Virginia. Virginia has the highest rate of births in the nation. A child born in Virginia born today will have an average of 12.3 years of life in the US.

------------
Typical:
The capital of the United States is Washington, which is difficult to describe. There is a cemetery for the dead situated in the suburbs of Carlsbad, the only town outside the San Ysidro Free Trade Zone with its own government department. It is fascinating to think that the magnificent Welsh landscape was created by Wright Brothers with hydraulic plates that were submerged in




---------
**We can see here that the results are pretty similar across each sampling type, althought not exactly. DC borders Virginia and a lot of government buildings are in Virginia, and Republicans do hold a large amount of power in DC, althought it definitely isnt the exact correct answer**

# 3. Prompting (50 pts)

In this problem, we will try various prompting approaches and prompt an LLM for a Math Reasoning Benchmark called [GSM8K](https://github.com/openai/grade-school-math), which contains grade school math word problems. This is a very common _reasoning_ benchmark used to test various LLMs.

The LLM that we will be using is [Google Gemini](https://gemini.google.com/). We will be prompting Gemini by using an API call to the Gemini Model. Normally, you can also prompt Open Source LLMs via the HuggingFace Library, however due to compute constraints, we use Gemini in this problem.

## Setting up the GSM8K Dataset and Google Gemini

Follow the steps below to download the GSM8K Dataset and to setup Google Gemini on Colab. You will automatically get points for this subpr

In [ ]:
from datasets import load_dataset

dataset = load_dataset("gsm8k", 'main')

In [ ]:
len(dataset['train']), len(dataset['test'])

(7473, 1319)

In [ ]:
# An example instance of this dataset

dataset['test'][6]

{'question': 'Toulouse has twice as many sheep as Charleston. Charleston has 4 times as many sheep as Seattle. How many sheep do Toulouse, Charleston, and Seattle have together if Seattle has 20 sheep?',
 'answer': 'If Seattle has 20 sheep, Charleston has 4 * 20 sheep = <<20*4=80>>80 sheep\nToulouse has twice as many sheep as Charleston, which is 2 * 80 sheep = <<2*80=160>>160 sheep\nTogether, the three has 20 sheep + 160 sheep + 80 sheep = <<20+160+80=260>>260 sheep\n#### 260'}

### Gemini Setup (from the official [Gemini documentation](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/get-started/python.ipynb))


Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel.

---

Give it the name `GEMINI_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GEMINI_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [ ]:
# All imports for this question
from google.colab import userdata
import google.generativeai as genai
from datasets import Dataset
import random
from typing import Callable, List, Any

In [ ]:
GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# Test if your setup is working, do not change the model name
model = genai.GenerativeModel("gemini-1.0-pro")
response = model.generate_content("What is Natural Language Processing? Explain it to a five year old.")
print(response.text)

Imagine you have a super smart computer friend named NLP. NLP can understand and talk to you in the same way you talk to your mom or dad.

Just like you learn new words and how to put them together to say sentences, NLP learns how to understand human language. When you say things to NLP, it listens carefully and tries to figure out what you mean.

Then, NLP can answer you back in a way that makes sense to you. It's like having a friend who speaks your language perfectly! That's what Natural Language Processing is all about. It's like giving computers the power to understand and talk to us in our own language.


## 3.1 Data and Prompting Setup (15 + 5 pts)

In this part, we will create some boilerplate code to process our dataset and generate prompts from the dataset.



### Processing the GSM8K Dataset

In [ ]:
def process_gsm8k_answers(dataset: Dataset) -> Dataset:
    """
    Processes the GSM8K dataset to remove reasoning chains and retain only the numerical answers.
    Assumes answers are separated from reasoning by the '###' string.

    Args:
    dataset (Dataset): Huggingface Dataset object for GSM8K.

    Returns:
    Dataset: Processed Dataset object with numerical answers only.
    """

    def extract_answer(sample):
        # IMPLEMENT HERE
        # Split the answer using '###' and return a dictionary with the key 'processed_answer'
        if '###' in sample['answer']:
            processed_answer = sample['answer'].split('###')[-1].strip()
        else:
            processed_answer = sample['answer'].strip()
        return {'processed_answer': processed_answer}

    return dataset.map(extract_answer)

In [ ]:
#TEST

gsm8k_test_processed = process_gsm8k_answers(dataset['test'])
print("Processed Test Dataset Example:")
print(gsm8k_test_processed[0])

Processed Test Dataset Example:
{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18', 'processed_answer': '# 18'}


### Building Prompts (15 pts)

We will be implementing FIVE (5) prompting methods. See their descriptions below -
1. **Zero-Shot Answer Only (2 pts)**: You prompt the model to only generate the answer to the question

2. **Zero-Shot Chain of Thought (CoT) (3 pts)**: Refer to the [Chain of Thought Paper](https://arxiv.org/abs/2201.11903). CoT refers to a reasoning chain that is generated by the model before generating the actual answer. This has shown to improve performance. In this setup, you will prompt the model to generate a reasoning chain before the answer.

3. **5-Shot Answer Only (2 pts)**: You provide some in-context examples to prompt the model with to generate the answer. This is analogous to Approach 1. Use the a random set of 5 examples from the training set to create the in-context examples.

4. **5-Shot CoT (3 pts)**: Combine Approaches 2 and 3 to do 5-shot CoT prompting.

5. **Your own prompt! (5 pts)**: Try something new. Think about how you solve Math problems and implement your own prompting method.

In [ ]:
def prompt_generation_zero_shot(problem: str) -> str:
    """
    Zero-shot prompt.

    Returns:
    str: The generated prompt.
    """
    # IMPLEMENT HERE
    return f"answer this math problem, I just want only the answer: \n\n Problem: {problem}\nAnswer:"


In [ ]:
def prompt_generation_zero_shot_cot(problem: str) -> str:
    """
    Zero-shot Chain of Thought (CoT) prompt.

    Returns:
    str: The generated prompt.
    """
    # IMPLEMENT HERE
    return f"answer the following math problem. First, think through the steps logically and then give an answer:\n\nProblem: {problem}\n\n Explanation and answer:"

In [ ]:
def prompt_generation_5_shot(problem: str, training_set: Dataset) -> str:
    """
    5-shot prompt generation for GSM8K problems. Randomly selects 5 examples from the training set.

    Returns:
    str: The generated prompt with 5 in-context_examples.
    """
    # IMPLEMENT HERE

    examples = random.sample(list(training_set), 5)
    examps = "\n\n".join(
        [f"Problem: {ex['question']}\nAnswer: {ex['processed_answer']}" for ex in examples]
    )
    return f"{examps}\n\n Problem: {problem}\nAnswer:"


In [ ]:
def prompt_generation_5_shot_wrapper(problem: str) -> str:
    t_set = process_gsm8k_answers(dataset['train'])
    return prompt_generation_5_shot(problem, t_set)

In [ ]:
def prompt_generation_5_shot_cot(problem: str, training_set: Dataset) -> str:
    """
    5-shot Chain of Thought (CoT) prompt generation. Randomly selects 5 examples
    from the training set and includes reasoning steps.

    Returns:
    str: The generated prompt with 5 CoT in-context examples.
    """
    # IMPLEMENT HERE
    examples = random.sample(training_set, 5)
    examps = "\n\n".join(
        [f"Problem: {ex['question']}\nReasoning: {ex['answer']}\nFinal Answer: {ex['processed_answer']}" for ex in examples]
    )

    return f"{examps}\n\n Problem: {problem}\n Reasoning and answer:"

In [ ]:
def prompt_generation_5_shot_cot_wrapper(problem: str) -> str:
    t_set = list(process_gsm8k_answers(dataset['train']))
    return prompt_generation_5_shot_cot(problem, t_set)

In [ ]:
# Feel free to change the method definition

def my_prompt(problem: str) -> str:
    """
    Your own unique way of prompting an LLM for Math word problems.

    Returns:
    str: The generated prompt
    """
    # IMPLEMENT HERE
    return f"solve the following math problem step by step. once you reach the answer, verify if its correct by checking your solution.\n\nProblem: {problem}\nSteps:\nAnswer:"

## 3.2 Prompting Gemini and Implementing Self-Consistency (5 + 5 + 10 pts)

Here, you will help build the wrapper for prompting Gemini using the prompt methods you have designed above.

You will then also implement Self-Consistency based prompting. Refer to the [Self-Consistency Paper](https://arxiv.org/abs/2203.11171). In order to implement Self-Consistency, you generate multiple Zero-Shot CoT (Approach 2 in the prompting methods) candidates, and take a majority vote of the answers predicted by each candidate.

### First, write the function where you will process the answer generated by the model. (5 pts)

Note that answer processing changes for different prompt types, so this function also takes in the name of the method in its argument.

In [ ]:
import re
def answer_processing(prediction: str, prompt_function: Any) -> str:
    """
    Processes the model's generated output to extract the final answer.

    Returns:
    str: The processed numerical answer.
    """

    prompt_name = prompt_function.__name__

    answer = None
    if "Answer:" in prediction:
        answer = prediction.split("Answer:")[-1].strip()
    elif "The answer is" in prediction:
        answer = prediction.split("The answer is")[-1].strip()
    else:
        answer = prediction.strip().split('\n')[-1].strip()

    num = re.findall(r"\d+", answer)
    if num:
        return f"# {''.join(num)}"
    else:
        return "# 0"

In [ ]:
# Do not change, method to calculate accuracy from predictions and ground truth labels

def evaluate_accuracy(predictions: List[str], ground_truths: List[str]) -> float:
    correct = 0
    total = len(predictions)

    for pred, true in zip(predictions, ground_truths):
        if pred == true:
            correct += 1

    accuracy = correct / total
    return accuracy * 100

### Next, write the wrapper function where you use all the building blocks constructed above to prompt the Gemini model (5 + 10 pts)


On how to prompt Gemini, refer to the [Gemini Text Generation Handbook](https://ai.google.dev/gemini-api/docs/text-generation?lang=python).

Hint: Reading this will help you figure out how to generate multiple candidates to implement Self-Consistency.

In [ ]:
def pipeline_generate(
    model_instance: Any,
    test_set: Dataset,
    prompt_function: Callable[[str], str],
    process_answer_function: Callable[[str], str],
    evaluation_function: Callable[[List[str], List[str]], float],
    self_consistency: int,
) -> float:
    """
    Args:
    model_instance (Any): The Google Gemini model instance.
    test_set (Dataset): The GSM8K test set to evaluate on.
    prompt_function (Callable): Function to generate prompts for the test set.
    process_answer_function (Callable): Function to process the model's generated answers.
    evaluation_function (Callable): Function to evaluate model's answers against the ground truth.
    self_consistency: Number of samples to run self-consistency approach on.
    If negative, 0 or 1, this implies regular prompting

    Returns:
    float: The accuracy of the model on the test set.
    """

    predictions = []
    grounds = []

    for sample in test_set:
        problem = sample["question"]
        ground = sample["processed_answer"]

        #print(f"Problem: {problem}")
        #print(f"ground answer: {ground}")

        if self_consistency > 1:
            answers = []
            for _ in range(self_consistency):
                prompt = prompt_function(problem)
                #print(f"\n{prompt}")
                response = model_instance.generate_content(prompt)
                #print(f"output:\n{response.text}")
                processed_answer = process_answer_function(response.text, prompt_function)
                #print(f"answer: {processed_answer}")
                answers.append(processed_answer)

            final_answer = max(set(answers), key=answers.count)
        else:
            prompt = prompt_function(problem)
            #print(f"\n{prompt}")
            response = model_instance.generate_content(prompt)
            #print(f"output:\n{response.text}")
            final_answer = process_answer_function(response.text, prompt_function)
            #print(f"final answer: {final_answer}")

        predictions.append(final_answer)
        grounds.append(ground)

    accuracy = evaluation_function(predictions, grounds)
    #print(f"Accuracy: {accuracy}%")
    return accuracy

In [ ]:
training_set = list(process_gsm8k_answers(dataset['train']))
print("Sample from the training set:", training_set[0])

Sample from the training set: {'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72', 'processed_answer': '# 72'}


In [ ]:
gsm8k_test_processed = process_gsm8k_answers(dataset['test'])

# The following line is just to test your systems, comment this line out to report results on the entire test set in 3.3
gsm8k_test_processed = Dataset.from_dict(gsm8k_test_processed[:5])

# Run model generation with zero-shot prompt generation
accuracy = pipeline_generate(
    model_instance=model,
    test_set=gsm8k_test_processed,
    prompt_function=prompt_generation_zero_shot_cot, # Change this to test different prompt methods
    process_answer_function=answer_processing,
    evaluation_function=evaluate_accuracy,
    self_consistency=5,
)

print(f"Accuracy: {accuracy}%")

Accuracy: 60.0%


## 3.3 Complete this table based on your implementation in 3.2 and answer the following questions (5 + 5 pts)

### Round each value up to two decimal points (5 pts)

Method|Accuracy
---|---|
0-shot| 80.00%
0-shot CoT| 80.00%
5-shot| 60.00%
5-shot CoT| 80.00%
My prompt| 20.00%
0-shot CoT Self-Consistency| 60.00%

### What was the intuition behind the prompt that you designed? (2 pts)

**The intuition behind it is that it forces chain of thought reasoning, where the model has to provide logical reasoning for each step of it's decisions. This, along with the verification request at the end, helps assure the process is logical and more accurate.**

### What are the merits and demerits of using advanced prompting approaches like Chain of Thought or Self-Consistency? (3 pts)

*   **Merits: More understanable to a user, rather than just giving an answer it gives intuition which justifies each step. This also improves accuracy, since the model can't hallucinate as much, it must have logic. This is how humans solve math problems so it correlates more to human use. Self consistency is able to find the most commonly appearing result, meaning it results from many possible approaches so its a strong candidate.**
  
*   **Demerits: For easy tasks, its a waste of resource and uses an unneccessary amount of computation. Self consistency can pose an issue if the model keeps messing up computations, skewing the most common results toward incorrect ones.**